# Generating Populations and Species
Assuming that the civilization is on the verge of moving from a phaze 0 to phase 1 civ.
* baseline species
* n populations (pops)
* population varience (within the norms of the species)
* allegiance groups to factions

Starting with an example user form data

In [32]:

from numpy import random, round
from sklearn.cluster import KMeans
import altair as alt
import sys, os
from sklearn.cluster import KMeans

import pandas as pd


In [46]:
# # TODO: figure out importing so that I can load game functions
# game_modules = '../../web/app/creators'
# django_root = '../..'
# print(os.listdir(django_root))
# sys.path.append(game_modules)
# sys.path.append(django_root)
# sys.path.append(language_p)
# import language


In [74]:

data = {
    'population_conformity':0.4,
    'population_literacy':0.7,
    'population_aggression':0.5,
    'population_constitution':0.4,
    'starting_pop': 100
}

def build_species(data):
    species = {}
    for attr in ['population_conformity', 'population_literacy', 'population_aggression', 'population_constitution']:
        species[attr] = data[attr]
    return species
    
species = build_species(data)
species


{'population_conformity': 0.4,
 'population_literacy': 0.7,
 'population_aggression': 0.5,
 'population_constitution': 0.4}

## Creating groups of populations, all with different attributes

Populations vary based on conformity, some populations are more literate, some are more aggressive. 

In [75]:
pop_std = .2 * (1-species['population_conformity'])
print("the population conformity: ", pop_std)

def vary_pops(species):
    pop = {}
    for k in list(species.keys()):
        pop[k] = abs(round(random.normal(species[k], pop_std),3))
    return pop

pops = pd.DataFrame([vary_pops(species) for i in range(data['starting_pop'])])
pops

the population conformity:  0.12


population_conformity  population_literacy  population_aggression  \
0                   0.618                0.670                  0.758   
1                   0.278                0.645                  0.512   
2                   0.486                0.665                  0.476   
3                   0.370                0.634                  0.462   
4                   0.375                0.679                  0.672   
..                    ...                  ...                    ...   
95                  0.434                0.776                  0.583   
96                  0.372                0.648                  0.415   
97                  0.239                0.432                  0.635   
98                  0.504                0.655                  0.413   
99                  0.480                0.769                  0.358   

    population_constitution  
0                     0.333  
1                     0.512  
2                     0.485  
3                     0.362  
4                     0.379  
..                      ...  
95                    0.466  
96                    0.575  
97                    0.228  
98                    0.498  
99                    0.429  

[100 rows x 4 columns]

In [79]:
kmeans = KMeans(n_clusters=2).fit(pops)
pops['faction_no'] = kmeans.labels_

In [80]:
kmeans.inertia_

3.71128204

In [81]:
chart = alt.Chart(pops).mark_circle().encode(x='population_literacy',y='population_aggression',color='faction_no:N')
chart

alt.Chart(...)

The end result is that population factions are grouped by the idealogical differences of the people in them. This gives each faction a distinct culture. 

In [84]:
pd.DataFrame(kmeans.cluster_centers_, columns=pops.columns)

population_conformity  population_literacy  population_aggression  \
0                0.40052              0.63176                0.57382   
1                0.40468              0.78764                0.45122   

   population_constitution  faction_no  
0                  0.33972         1.0  
1                  0.44340         0.0

The cluster centers can represent the 'zeitgeist' of that faction's culture. 

## Deciding the right number of factions to have.

In [ ]:
#